In [1]:
try:
  import geopandas as gpd
except:
  !pip install geopandas
  import geopandas as gpd

import pandas as pd

In [3]:
# Leer el GeoJSON de los geohashes de Ecuador
geohashes_gdf = gpd.read_file('ecuador_geohashes_nivel5.geojson')[['geohash','geometry']]

In [4]:
geohashes_gdf.head()

,geohash,geometry
0,d0p4y,"POLYGON ((-79.89258 0.4834, -79.89258 0.52734,..."
1,3zyym,"POLYGON ((-91.49414 -0.30762, -91.53809 -0.307..."
2,6r2rs,"POLYGON ((-78.22266 -2.90039, -78.22266 -2.856..."
3,6r9z2,"POLYGON ((-76.28906 -1.53809, -76.28906 -1.494..."
4,6rbdy,"POLYGON ((-77.7832 -0.92285, -77.7832 -0.87891..."


In [5]:
# Leer el shapefile de los cantones de Ecuador
cantones_gdf = gpd.read_file('nxcantones.shp')

In [6]:
cantones_gdf.head()

,DPA_VALOR,DPA_ANIO,DPA_CANTON,DPA_DESCAN,DPA_PROVIN,DPA_DESPRO,geometry
0,0,2012,0201,GUARANDA,02,BOLIVAR,"POLYGON ((710807.591 9872762.982, 710853.291 9..."
1,0,2012,0202,CHILLANES,02,BOLIVAR,"POLYGON ((720530.21 9793835.659, 720541.38 979..."
2,0,2012,0203,CHIMBO,02,BOLIVAR,"POLYGON ((711141.911 9822095.666, 711185.441 9..."
3,0,2012,0204,ECHEANDIA,02,BOLIVAR,"POLYGON ((700585.537 9836991.327, 700609.658 9..."
4,0,2012,0205,SAN MIGUEL,02,BOLIVAR,"POLYGON ((721746.267 9813431.09, 721746.237 98..."


In [7]:
# Asegurarse de que ambos GeoDataFrames estén en el mismo sistema de coordenadas
geohashes_gdf = geohashes_gdf.to_crs('EPSG:4326')
cantones_gdf = cantones_gdf.to_crs('EPSG:4326')

In [8]:
# Reproyectar a un sistema de coordenadas proyectado para calcular áreas correctamente
geohashes_gdf = geohashes_gdf.to_crs('EPSG:32717')  # UTM zone 17S
cantones_gdf = cantones_gdf.to_crs('EPSG:32717')

In [9]:
# Realizar una intersección espacial para obtener las áreas de superposición
interseccion = gpd.overlay(geohashes_gdf, cantones_gdf, how='intersection')

<ipython-input-9-c768e638fea2>:2: UserWarning: `keep_geom_type=True` in overlay resulted in 5 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  interseccion = gpd.overlay(geohashes_gdf, cantones_gdf, how='intersection')


In [10]:
# Calcular el área de superposición para cada intersección
interseccion['area_superposicion'] = interseccion.geometry.area

In [11]:
interseccion.head()

,geohash,DPA_VALOR,DPA_ANIO,DPA_CANTON,DPA_DESCAN,DPA_PROVIN,DPA_DESPRO,geometry,area_superposicion
0,d0p4y,0,2012,0803,MUISNE,08,ESMERALDAS,"POLYGON ((623230.867 10058298.307, 628121.631 ...",2.376054e+07
1,3zyym,0,2012,2002,ISABELA,20,GALAPAGOS,"POLYGON ((-679321.242 9965411.719, -679323.528...",1.489178e+07
2,6r2rs,0,2012,1405,SANTIAGO,14,MORONA SANTIAGO,"POLYGON ((808788.566 9683901.161, 813678.389 9...",2.377994e+07
3,6r9z2,0,2012,1604,ARAJUNO,16,PASTAZA,"POLYGON ((1024626.982 9834288.851, 1029532.16 ...",2.390781e+07
4,6rbdy,0,2012,1503,ARCHIDONA,15,NAPO,"POLYGON ((858096.478 9902700.055, 862993.791 9...",2.382584e+07


In [12]:
# Para cada geohash, encontrar el cantón con el área de superposición máxima
# Primero, agrupamos por 'geohash' y obtenemos el área máxima
max_area = interseccion.groupby('geohash')['area_superposicion'].max().reset_index()

In [13]:
# Unimos esta información con el DataFrame de intersección para obtener el cantón correspondiente
interseccion = interseccion.merge(max_area, on=['geohash', 'area_superposicion'])

In [14]:
interseccion.head()

,geohash,DPA_VALOR,DPA_ANIO,DPA_CANTON,DPA_DESCAN,DPA_PROVIN,DPA_DESPRO,geometry,area_superposicion
0,d0p4y,0,2012,0803,MUISNE,08,ESMERALDAS,"POLYGON ((623230.867 10058298.307, 628121.631 ...",2.376054e+07
1,3zyym,0,2012,2002,ISABELA,20,GALAPAGOS,"POLYGON ((-679321.242 9965411.719, -679323.528...",1.489178e+07
2,6r2rs,0,2012,1405,SANTIAGO,14,MORONA SANTIAGO,"POLYGON ((808788.566 9683901.161, 813678.389 9...",2.377994e+07
3,6r9z2,0,2012,1604,ARAJUNO,16,PASTAZA,"POLYGON ((1024626.982 9834288.851, 1029532.16 ...",2.390781e+07
4,6rbdy,0,2012,1503,ARCHIDONA,15,NAPO,"POLYGON ((858096.478 9902700.055, 862993.791 9...",2.382584e+07


In [15]:
# Ahora, seleccionamos las columnas relevantes
resultado = interseccion[['geohash', 'DPA_PROVIN', 'DPA_DESPRO', 'DPA_CANTON', 'DPA_DESCAN', 'geometry']]

In [16]:
resultado.head()

,geohash,DPA_PROVIN,DPA_DESPRO,DPA_CANTON,DPA_DESCAN,geometry
0,d0p4y,08,ESMERALDAS,0803,MUISNE,"POLYGON ((623230.867 10058298.307, 628121.631 ..."
1,3zyym,20,GALAPAGOS,2002,ISABELA,"POLYGON ((-679321.242 9965411.719, -679323.528..."
2,6r2rs,14,MORONA SANTIAGO,1405,SANTIAGO,"POLYGON ((808788.566 9683901.161, 813678.389 9..."
3,6r9z2,16,PASTAZA,1604,ARAJUNO,"POLYGON ((1024626.982 9834288.851, 1029532.16 ..."
4,6rbdy,15,NAPO,1503,ARCHIDONA,"POLYGON ((858096.478 9902700.055, 862993.791 9..."


In [17]:
resultado[['geohash','DPA_CANTON']].groupby('geohash').count().sort_values(by='DPA_CANTON',ascending=False)

,DPA_CANTON
geohash,
3zyfp,1
6rbhp,1
6rbje,1
6rbjf,1
6rbjg,1
...,...
6pz37,1
6pz38,1
6pz39,1


In [18]:
geohashes_mapeados = geohashes_gdf.merge(resultado[['geohash','DPA_PROVIN', 'DPA_DESPRO', 'DPA_CANTON', 'DPA_DESCAN']], on='geohash', how='left')
geohashes_mapeados.head()

,geohash,geometry,DPA_PROVIN,DPA_DESPRO,DPA_CANTON,DPA_DESCAN
0,d0p4y,"POLYGON ((623231.695 10053440.113, 623230.867 ...",08,ESMERALDAS,0803,MUISNE
1,3zyym,"POLYGON ((-674347.189 9965416.677, -679321.242...",20,GALAPAGOS,2002,ISABELA
2,6r2rs,"POLYGON ((808776.718 9679038.039, 808788.566 9...",14,MORONA SANTIAGO,1405,SANTIAGO
3,6r9z2,"POLYGON ((1024616.355 9829414.986, 1024626.982...",16,PASTAZA,1604,ARAJUNO
4,6rbdy,"POLYGON ((858092.179 9897835.052, 858096.478 9...",15,NAPO,1503,ARCHIDONA


In [19]:
geohashes_mapeados.to_file('geohashes_mapeados_con_cantones.geojson', driver='GeoJSON')